In [ ]:
!pip install duckdb

In [54]:
import duckdb
import pandas as pd
from pathlib import Path

calendar_path = Path("./calendar.csv")
data_path = Path("./data.csv")

calendar_df = pd.read_csv(calendar_path)
data_df = pd.read_csv(data_path)

In [ ]:
# I was casting dates in python, but decided to keep everything in SQL query
# data_df['MARKET_DATE'] = pd.to_datetime(data_df['MARKET_DATE'], format="%Y-%m-%d")

# calendar_df = calendar_df[calendar_df['DATE'] < '2030-01-01']
# calendar_df['DATE'] = pd.to_datetime(calendar_df['DATE'], format="%Y-%m-%d")

calendar_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146097 entries, 0 to 146096
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   DATE                146097 non-null  object
 1   DAY_OF_WEEK         146097 non-null  int64 
 2   TEXT_DAY_OF_WEEK    146097 non-null  object
 3   TEXT_MONTH_OF_YEAR  146097 non-null  object
 4   IS_HOLIDAY          146097 non-null  bool  
 5   IS_WORKING_DAY      146097 non-null  bool  
dtypes: bool(2), int64(1), object(3)
memory usage: 4.7+ MB


In [60]:
duckdb.sql("""

    SELECT
        CAST(DATE AS DATE) as date,
        IS_HOLIDAY,
        IS_WORKING_DAY
    FROM calendar_df
    """).df()

,date,IS_HOLIDAY,IS_WORKING_DAY
0,1900-01-01,True,True
1,1900-01-02,False,True
2,1900-01-03,False,True
3,1900-01-04,False,True
4,1900-01-05,False,True
...,...,...,...
146092,2299-12-27,False,True
146093,2299-12-28,False,True
146094,2299-12-29,False,True
146095,2299-12-30,False,False


In [ ]:
duckdb.sql("""
-- CREATE OR REPLACEVIEW weekly_returns AS
(
    WITH casted_dates as (
        SELECT
            CAST(DATE AS DATE) as DATE,
            IS_HOLIDAY,
            IS_WORKING_DAY
        FROM calendar_df
    )
    , business_days as (
        SELECT
            DATE,
            WEEK(DATE) as week_number,
            MIN(DATE) over (partition by WEEK(DATE), YEAR(DATE)) as first_day_of_week,
            max(DATE) over (partition by WEEK(DATE), YEAR(DATE)) as last_day_of_week,
        FROM casted_dates
        WHERE True
            and IS_HOLIDAY = FALSE
            and IS_WORKING_DAY = TRUE
    )
    SELECT
        b.week_number,
        b.first_day_of_week,
        b.last_day_of_week,
        CAST(( EXP( SUM( LN(1 + DAILY_RETURN / 100))) - 1)*100 AS DECIMAL(10,2)) || ' %' as return_percentage
    FROM data_df d
    JOIN business_days b on cast(d.MARKET_DATE as DATE) = b.DATE
    GROUP BY 1,2,3
    ORDER BY b.week_number DESC
)
""").df()



,week_number,first_day_of_week,last_day_of_week,return_percentage
0,9,2025-02-24,2025-02-28,10.38 %
1,8,2025-02-17,2025-02-21,10.73 %
2,7,2025-02-10,2025-02-14,8.68 %
3,6,2025-02-03,2025-02-07,9.53 %
4,5,2025-01-27,2025-01-31,11.66 %
5,4,2025-01-20,2025-01-24,11.15 %
6,3,2025-01-13,2025-01-17,11.18 %
7,2,2025-01-06,2025-01-10,9.19 %
8,1,2025-01-02,2025-12-30,4.26 %
